# Init

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device, torch.cuda.current_device(), torch.cuda.get_device_name(0))

cuda 0 GeForce GTX 1050


In [3]:
# hyper parameters
epochs = 20
batch_size = 4
learning_rate = 0.001

# CIFAR10 Dataset

In [4]:
# CIFAR10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())

Files already downloaded and verified


In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # convulational layers
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # fully connected layers
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # conv1 with relu
        x = self.pool(x)
        x = F.relu(self.conv2(x))  # conv2 with relu
        x = self.pool(x)
        # flatten images before fc layers
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training

In [7]:
# model
model = ConvNet().to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# training loop
print('Started Training...')
n_total_steps = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32]
        # input layer: 3 in channels, 6 out channels, 5 kernels
        images = images.to(device)
        labels = labels.to(device)

        # main 5
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1}/{epochs}, step {i+1}/{n_total_steps}: loss = {loss.item():.4f}')

print('Finished Training...')

Started Training...
epoch 1/20, step 100/12500: loss = 2.2701
epoch 1/20, step 200/12500: loss = 2.2753
epoch 1/20, step 300/12500: loss = 2.3085
epoch 1/20, step 400/12500: loss = 2.2906
epoch 1/20, step 500/12500: loss = 2.2786
epoch 1/20, step 600/12500: loss = 2.2631
epoch 1/20, step 700/12500: loss = 2.3347
epoch 1/20, step 800/12500: loss = 2.3408
epoch 1/20, step 900/12500: loss = 2.2379
epoch 1/20, step 1000/12500: loss = 2.3175
epoch 1/20, step 1100/12500: loss = 2.2655
epoch 1/20, step 1200/12500: loss = 2.2974
epoch 1/20, step 1300/12500: loss = 2.2990
epoch 1/20, step 1400/12500: loss = 2.3074
epoch 1/20, step 1500/12500: loss = 2.3047
epoch 1/20, step 1600/12500: loss = 2.2918
epoch 1/20, step 1700/12500: loss = 2.3005
epoch 1/20, step 1800/12500: loss = 2.2929
epoch 1/20, step 1900/12500: loss = 2.2897
epoch 1/20, step 2000/12500: loss = 2.3062
epoch 1/20, step 2100/12500: loss = 2.3181
epoch 1/20, step 2200/12500: loss = 2.3135
epoch 1/20, step 2300/12500: loss = 2.3223


# Testing

In [8]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        # value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the Model: {acc:.2f} %')

    for i in range(10):
        acc = 100.0 * float(n_class_correct[i] / n_class_samples[i])
        print(f' Accuracy of {classes[i]}: {acc:.2f} %')
        

Accuracy of the Model: 57.82 %
 Accuracy of plane: 57.70 %
 Accuracy of car: 66.00 %
 Accuracy of bird: 55.70 %
 Accuracy of cat: 32.20 %
 Accuracy of deer: 48.40 %
 Accuracy of dog: 42.50 %
 Accuracy of frog: 73.60 %
 Accuracy of horse: 62.70 %
 Accuracy of ship: 79.10 %
 Accuracy of truck: 60.30 %
